In [1]:
from pathlib import Path
import os, copy, sys
notebook_path = Path().absolute()
project_root = notebook_path.parent
sys.path.append(str(project_root))

# Then update the assets path
path = project_root / 'assets/crossword_data'

from crossword import Variable, Crossword
from kenken import Cage, Kenken
from custom_classes import Overlap, VectorSlice, Vector
from crossword_creators.pcoster import CrosswordCreator

from kenken import Cage, Kenken
from typing import *


In [2]:
############################
@overload
def revise(
    x: Cage, y: Cage,
    domains: Dict[Cage, Set[Vector]],
    overlaps: Dict[Tuple[Cage, Cage], Overlap],
    test_type: str
    ) -> bool:
    ...

def revise(
    x: Variable, y: Variable,
    domains: Dict[Variable, Set[str]],
    overlaps: Dict[Tuple[Variable, Variable], Tuple[int, int]],
    test_type: str) -> bool:

    # Validate 'test_type' first
    assert test_type in ['crossword', 'kenken'], (f'test_type "{test_type}" not allowed. Must be "crossword" or "kenken".')
    if test_type == 'crossword':
        assert isinstance(x, Variable) and isinstance(y, Variable), ('x and y must be of type Variable')
        assert isinstance(domains, dict), (f'domains should be of type dict, got {type(domains).__name__}')
        assert isinstance(overlaps, dict), (f'overlaps should be of type dict, got {type(overlaps).__name__}')

        for variable, domain in domains.items():
            assert isinstance(variable, Variable), (f'Key "{variable}" must be of type Variable')
            assert isinstance(domain, set), (f'Domain for "{variable}" must be a set, got {type(domain).__name__}')
            assert domain, (f'Domain for "{variable}" must not be empty.')
            for word in domain:
                assert isinstance(word, str), (f'Word "{word}" in {variable} is not a string')
                assert word, f'Blank word found in {variable}'
                assert len(word) > 2, (f'Word "{word}" in {variable} must have length >= 3')
    else: #test_type == 'kenken':
        assert isinstance(x, Cage) and isinstance(y, Cage), ('x and y must be of type Cage')
        assert isinstance(domains, dict), (f'domains should be of type dict, got {type(domains).__name__}')
        assert isinstance(overlaps, dict), (f'overlaps should be of type dict, got {type(overlaps).__name__}')
        for cage, dom in domains.items():
            assert isinstance(cage, Cage), (f'Key "{cage}" must be of type Cage')
            assert isinstance(dom, set), (f'Domain for "{cage}" must be a set, got {type(dom).__name__}')
            assert dom, (f'Domain for "{cage}" must not be empty.')
            for vector in dom:
                assert isinstance(vector, Vector), (f'Vector "{vector}" in cage #{cage.id} is not of type Vector')
                assert len(vector) > 0, (f'Empty vector found in cage #{cage.id}')
   
   
    def overlap_satisfied(x, y, val_x, val_y):
        olap = overlaps.get((x,y))
        if not olap:
            return True
        x_index, y_index = overlaps[x,y]
        return True if val_x[x_index] == val_y[y_index] else False

    revision = False
    to_remove = set()

    # Iterate over domain of x and y, track any inconsistent x:
    for val_x in domains[x]:
        consistent = False
        for val_y in domains[y]:
            if val_x != val_y and overlap_satisfied(x, y, val_x, val_y):
                consistent = True
                break
        if not consistent:
            to_remove.add(val_x)
            revision = True
    # Remove any domain variables that aren't arc consistent:
    domains[x] = domains[x] - to_remove
    return revision
###


In [3]:
path = project_root / 'assets/crossword_data'
file_number = 1
structure_file = path / f'structure{file_number}.txt'
words_file = path / f'words{file_number}.txt'


def get_revision(revise_f, crossword, creator, var_idx1, var_idx2):

    domains = creator.domains
    get_var = lambda id: next((key for key in domains.keys() if key.id == id), None)
    creator.enforce_node_consistency()
    arcs = [(x,y) for x in domains for y in domains if x!=y and x.direction!=y.direction and crossword.overlaps[x,y] is not None]
    x = get_var(var_idx1)
    y = get_var(var_idx2)
    lenx_before = len(domains[x])
    if revise_f is None:
        was_revised = creator.revise(x,y)
    else:
        was_revised = revise(x=x,y=y, domains=domains, overlaps=crossword.overlaps, test_type='crossword')
    lenx_after = len(domains[x])
    #print(f'x: {lenx_before}-->{lenx_after}')
    return was_revised, (lenx_before, lenx_after)

# get size
from crossword import Crossword
size = len(Crossword(structure_file, words_file).variables)


In [4]:
import importlib

creator_names = [
    "baseline",
    "pcoster", 
    "verano_20", 
    "hadeeer98", 
    "chezslice", 
    "iron8kid", 
    "marcoshernanz"
]

def run_creator(structure_file, words_file, size, get_revision):
    results = {}
    for mod_name in creator_names:
        # Dynamically import the module
        module = importlib.import_module(f"crossword_creators.{mod_name}")
        CrosswordCreator = module.CrosswordCreator

        reviseds = []
        pairs = set()
        for i in range(size):
            for j in range(size):
                if i != j:
                    crossword = Crossword(structure_file, words_file)
                    creator = CrosswordCreator(crossword)
                    revised, (b, a) = get_revision(
                        revise_f=None,
                        crossword=crossword,
                        creator=creator,
                        var_idx1=i,
                        var_idx2=j
                    )
                    reviseds.append(revised)
                    pairs.add((b, a))

        results[mod_name] = (reviseds, pairs)

    return results



print('Compararing the the net effect of revision on a single pair of variables.')
for file_number in [0,1,2,3]:
    structure_file = path / f'structure{file_number}.txt'
    words_file = path / f'words{file_number}.txt'
    size = len(Crossword(structure_file, words_file).variables)
    results = run_creator(structure_file, words_file, size, get_revision)
    print(f'crossword {file_number}')
    print(f'Change in domain size (before, after) for revision of a single ordered pairs of variables:')
    for k,v in results.items():
        print(f'{k:15} {v[1]}')
    print()
    #print(f'single ordered pairs: revised or not:()')
    #for k, v in results.items():
    #    print(f'{k:15} {v[0]}')    
    print(f'\n\n-------------------')


Compararing the the net effect of revision on a single pair of variables.
crossword 0
Change in domain size (before, after) for revision of a single ordered pairs of variables:
baseline        {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}
pcoster         {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}
verano_20       {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}
hadeeer98       {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}
chezslice       {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}
iron8kid        {(4, 4), (3, 3)}
marcoshernanz   {(4, 4), (4, 3), (3, 1), (3, 3), (3, 2)}



-------------------
crossword 1
Change in domain size (before, after) for revision of a single ordered pairs of variables:
baseline        {(4, 4), (5, 5), (4, 3), (5, 1), (10, 10), (10, 3), (5, 3), (4, 1), (5, 2)}
pcoster         {(4, 4), (5, 5), (4, 3), (5, 1), (10, 10), (10, 3), (5, 3), (4, 1), (5, 2)}
verano_20       {(4, 4), (5, 5), (4, 3), (5, 1), (10, 10), (10, 3), (5, 3), (4, 1), (5, 2)}
hadeeer98       {(4, 4), (5, 5), (4, 3), (5, 1

In [5]:
creator_names = ['baseline', 'chezslice', 'hadeeer98', 'iron8kid', 'marcoshernanz', 'pcoster', 'verano_20']

def ac3(file_number):
    structure_file = path / f'structure{file_number}.txt'
    words_file = path / f'words{file_number}.txt'
    results = {}
    for mod_name in creator_names:
        # Dynamically import the module
        module = importlib.import_module(f"crossword_creators.{mod_name}")
        CrosswordCreator = module.CrosswordCreator
        crossword = Crossword(structure_file, words_file)
        creator = CrosswordCreator(crossword)
        creator.enforce_node_consistency()
        pre_ac3_lengths = [len(creator.domains[var]) for var in sorted(creator.domains, key=lambda var: var.id)]
        success = creator.ac3()
        if success:
            post_ac3_lengths = [len(creator.domains[var]) for var in sorted(creator.domains, key=lambda var: var.id)]
            domain_reduction = zip(pre_ac3_lengths, post_ac3_lengths)
        else:
            domain_reduction = 'ac3 failed'
        results[mod_name] = domain_reduction
    return results

for i in [0,1,2]:
    print(f'crossword {i}')
    ac3_results = ac3(i)
    for name, result in ac3_results.items():
        if isinstance(result, str):
            print(f'{name:15}: {result}')
        else:
            print(f'{name:15}: {list(result)}')  # Fixed indentation - moved inside the for loop

    
    print('\n\n------------')



crossword 0
baseline       : [(3, 1), (4, 1), (3, 1), (3, 1)]
chezslice      : ac3 failed
hadeeer98      : [(3, 1), (4, 1), (3, 2), (3, 1)]
iron8kid       : [(3, 3), (4, 4), (3, 3), (3, 3)]
marcoshernanz  : [(3, 1), (4, 1), (3, 2), (3, 1)]
pcoster        : [(3, 1), (4, 1), (3, 1), (3, 1)]
verano_20      : [(3, 1), (4, 1), (3, 1), (3, 1)]


------------
crossword 1
baseline       : [(5, 1), (5, 2), (10, 1), (4, 1), (10, 1), (5, 2)]
chezslice      : ac3 failed
hadeeer98      : [(5, 1), (5, 2), (10, 1), (4, 1), (10, 1), (5, 2)]
iron8kid       : [(5, 5), (5, 5), (10, 10), (4, 4), (10, 10), (5, 5)]
marcoshernanz  : [(5, 1), (5, 2), (10, 1), (4, 1), (10, 1), (5, 2)]
pcoster        : [(5, 1), (5, 2), (10, 1), (4, 1), (10, 1), (5, 2)]
verano_20      : [(5, 1), (5, 2), (10, 1), (4, 1), (10, 1), (5, 2)]


------------
crossword 2
baseline       : [(491, 414), (502, 500), (502, 500), (483, 382), (502, 490), (174, 169)]
chezslice      : ac3 failed
hadeeer98      : [(491, 414), (502, 502), (502, 50